In [ ]:
from pynq import Overlay
from pynq import MMIO
import time

def bcd_to_int(bcd):
    return ((bcd >> 4) * 10) + (bcd & 0x0F)

In [ ]:
ol = Overlay('top_wrapper.xsa')

In [ ]:
ol.ip_dict.keys()

In [ ]:
ol.ip_dict['axi_uartlite_0']

In [ ]:
class UartAXI:
    RX_FIFO = 0x00
    TX_FIFO = 0x04
    STAT_REG = 0x08
    CTRL_REG = 0x0C

    RX_VALID = 0
    RX_FULL = 1
    TX_EMPTY = 2
    TX_FULL = 3
    IS_INTR = 4
    OVERRUN_ERR = 5
    FRAME_ERR = 6
    PARITY_ERR = 7

    RST_TX = 0
    RST_RX = 1
    INTR_EN = 4

    def __init__(self, address):
        self.uart = MMIO(address, 0x10000, debug=False)
        self.address = address

    def setupCtrlReg(self):
        # Reset FIFO
        self.uart.write(self.CTRL_REG, (1 << self.RST_TX) | (1 << self.RST_RX))
        time.sleep(0.001)
        self.uart.write(self.CTRL_REG, 0)

    def read(self, count, timeout=10):
        buf = bytearray()
        stop_time = time.time() + timeout
        for _ in range(count):
            while not (self.uart.read(self.STAT_REG) & (1 << self.RX_VALID)) and time.time() < stop_time:
                pass
            if time.time() >= stop_time:
                break
            buf.append(self.uart.read(self.RX_FIFO) & 0xFF)
        return bytes(buf)

    def write(self, buf, timeout=10):
        if isinstance(buf, str):
            buf = buf.encode()

        stop_time = time.time() + timeout
        wr_count = 0
        for b in buf:
            while not (self.uart.read(self.STAT_REG) & (1 << self.TX_EMPTY)):
                if time.time() > stop_time:
                    break
            self.uart.write(self.TX_FIFO, b & 0xFF)  # 確保寫入 8-bit
            wr_count += 1
        return wr_count


In [ ]:
address = 0x42C00000
uart = UartAXI(address)
# Setup AXI UART register
uart.setupCtrlReg()

In [ ]:
uart.write('HELLO\r\n')